In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from typing import List

import pandas as pd
from artifact_experiment.binary_classification import (
    BinaryClassificationArrayCollectionType,
    BinaryClassificationArrayType,
    BinaryClassificationPlan,
    BinaryClassificationPlotCollectionType,
    BinaryClassificationPlotType,
    BinaryClassificationScoreCollectionType,
    BinaryClassificationScoreType,
    BinaryClassSpec,
)
from artifact_experiment.tracking import (
    ClearMLTrackingClient,
    FilesystemTrackingClient,
    MlflowTrackingClient,
    NeptuneTrackingClient,
)
from scipy.special import logit as logt_fn

# Source

In [ ]:
artifact_experiment_root = Path().absolute().parent

df_classification_results = pd.read_csv(
    artifact_experiment_root / "assets/binary_classification.csv"
)

df_classification_results

In [ ]:
true = df_classification_results["arthritis_true"].to_dict()
predicted = df_classification_results["arthritis_pred"].to_dict()
probs_pos = df_classification_results["arthritis_prob_est"].to_dict()
logits = df_classification_results["arthritis_prob_est"].apply(logt_fn).to_dict()

In [ ]:
class_spec = BinaryClassSpec(class_names=["0", "1"], positive_class="1")

class_spec

# Validation Plan

In [ ]:
class MyValidationPlan(BinaryClassificationPlan):
    @staticmethod
    def _get_score_types() -> List[BinaryClassificationScoreType]:
        return [
            BinaryClassificationScoreType.PRECISION,
            BinaryClassificationScoreType.RECALL,
            BinaryClassificationScoreType.ROC_AUC,
            BinaryClassificationScoreType.PR_AUC,
        ]

    @staticmethod
    def _get_array_types() -> List[BinaryClassificationArrayType]:
        return []

    @staticmethod
    def _get_plot_types() -> List[BinaryClassificationPlotType]:
        return [
            BinaryClassificationPlotType.SCORE_PDF,
            BinaryClassificationPlotType.GROUND_TRUTH_PROB_PDF,
        ]

    @staticmethod
    def _get_score_collection_types() -> List[BinaryClassificationScoreCollectionType]:
        return [
            BinaryClassificationScoreCollectionType.SCORE_MEANS,
            BinaryClassificationScoreCollectionType.SCORE_STDS,
            BinaryClassificationScoreCollectionType.SCORE_STATS,
        ]

    @staticmethod
    def _get_array_collection_types() -> List[BinaryClassificationArrayCollectionType]:
        return []

    @staticmethod
    def _get_plot_collection_types() -> List[BinaryClassificationPlotCollectionType]:
        return [
            BinaryClassificationPlotCollectionType.THRESHOLD_VARIATION_CURVES,
            BinaryClassificationPlotCollectionType.CONFUSION_MATRIX_PLOTS,
        ]

In [ ]:
plan = MyValidationPlan.create(resource_spec=class_spec)

In [ ]:
plan.execute_classifier_evaluation(true=true, predicted=predicted, probs_pos=probs_pos)

# Filesystem Logging

In [ ]:
filesystem_tracking_client = FilesystemTrackingClient.build(experiment_id="demo")

In [ ]:
plan = MyValidationPlan.create(resource_spec=class_spec, tracking_client=filesystem_tracking_client)

In [ ]:
plan.execute_classifier_evaluation(true=true, predicted=predicted, probs_pos=probs_pos)

In [ ]:
filesystem_tracking_client.run.stop()

# ClearML Logging

In [ ]:
CLEAR_ML_PROJECT_NAME = "Artifact-ML demo"


clear_ml_client = ClearMLTrackingClient.build(experiment_id=CLEAR_ML_PROJECT_NAME)

In [ ]:
plan = MyValidationPlan.create(resource_spec=class_spec, tracking_client=clear_ml_client)

In [ ]:
plan.execute_classifier_evaluation(true=true, predicted=predicted, probs_pos=probs_pos)

In [ ]:
clear_ml_client.run.stop()

# Neptune Logging

In [ ]:
NEPTUNE_PROJECT_NAME = "h.papoulias/privacy-research-longitudinal"

neptune_client = NeptuneTrackingClient.build(experiment_id=NEPTUNE_PROJECT_NAME)

In [ ]:
plan = MyValidationPlan.create(resource_spec=class_spec, tracking_client=neptune_client)

In [ ]:
plan.execute_classifier_evaluation(true=true, predicted=predicted, probs_pos=probs_pos)

In [ ]:
neptune_client.run.stop()

# Mlflow Logging

In [ ]:
MLFLOW_EXPERIMENT_ID = "Artifact-ML demo"

mlflow_client = MlflowTrackingClient.build(experiment_id=MLFLOW_EXPERIMENT_ID)

In [ ]:
plan = MyValidationPlan.create(resource_spec=class_spec, tracking_client=mlflow_client)

In [ ]:
plan.execute_classifier_evaluation(true=true, predicted=predicted, probs_pos=probs_pos)

In [ ]:
mlflow_client.run.stop()